In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# Load your Excel file
file_path = 'Cases_Report___General_Analysis - 2024-08-27.xlsx'
df = pd.read_excel(file_path)
# Convert 'Date Closed' to datetime and extract the year
df['Date Closed'] = pd.to_datetime(df['Date Closed'])
df['Year Closed'] = df['Date Closed'].dt.year

In [3]:
bins = [0, 17, 24, 34, 44, 54, 64, 80, 120]
labels = ['1-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65-80', '80+']

In [4]:
# Bin the data
df['Age Group'] = pd.cut(df['Age at Intake'], bins=bins, labels=labels, right=True)

In [5]:
df['Case Close Reason'].value_counts()

Case Close Reason
A Counsel and Advice                                             81509
H Administrative Agency Decision                                 18939
B Limited Action (was Brief Service)                             18017
G Negotiated Settlement (with Litigation)                         3786
Xa Client Withdrew before Legal Assistance                        2860
F Negotiated Settlement (without Litigation)                      2825
Ia Uncontested Court Decisions                                    2723
K Other                                                           2715
L Extensive Service (not FGHI)                                    2377
Ib Contested Court Decisions                                      1878
C Referred After Legal Assessment(Pre2009)                        1614
I Court Decision(Pre2009)                                          854
Xe Not timely closed                                               284
D Insufficient Merit to Proceed(Pre2009)                   

In [6]:
# List of values to filter by
case_close_reasons = [
    'A Counsel and Advice',
    'H Administrative Agency Decision',
    'B Limited Action (was Brief Service)',
    'G Negotiated Settlement (with Litigation)',
    'F Negotiated Settlement (without Litigation)',
    'Ia Uncontested Court Decisions',
    'L Extensive Service (not FGHI)',
    'Ib Contested Court Decisions',
    'C Referred After Legal Assessment(Pre2009)',
    'I Court Decision(Pre2009)',
    'D Insufficient Merit to Proceed(Pre2009)',
    'J Change in Eligibility Status(Pre2009)',
    'Ic Appeals'
]

# Filter the DataFrame
new_df = df[df['Case Close Reason'].isin(case_close_reasons)]

# Count the occurrences of each 'Case Close Reason'
case_close_reason_counts = new_df['Case Close Reason'].value_counts()
print(case_close_reason_counts)

Case Close Reason
A Counsel and Advice                            81509
H Administrative Agency Decision                18939
B Limited Action (was Brief Service)            18017
G Negotiated Settlement (with Litigation)        3786
F Negotiated Settlement (without Litigation)     2825
Ia Uncontested Court Decisions                   2723
L Extensive Service (not FGHI)                   2377
Ib Contested Court Decisions                     1878
C Referred After Legal Assessment(Pre2009)       1614
I Court Decision(Pre2009)                         854
D Insufficient Merit to Proceed(Pre2009)          231
J Change in Eligibility Status(Pre2009)            49
Ic Appeals                                         32
Name: count, dtype: int64


In [7]:
new_df.head()

,ID#,Client First Name,Client Last Name,Disposition,Legal Problem,Special Legal Problem Code,Intake Date,Date Opened,Date Closed,Case Close Reason,Funding Code(s),Outcome,Age at Intake,Gender,Race,AMI Category,Program Name,Total Number Helped,Year Closed,Age Group
0,24-0455696,Kristen,McDaniel,Closed,01 Bankruptcy/Debtor Relief,010 Bankruptcy: Unsecured Deb,08/07/2024,2024-08-14,2024-08-14,A Counsel and Advice,14 IOTA,09-09 - Obtained Case Referral on a Consumer M...,48,Female,White,< 30% of Area Median Income (AMI),Intake Dept,1,2024,45-54
1,24-0455696,Kristen,McDaniel,Closed,01 Bankruptcy/Debtor Relief,010 Bankruptcy: Unsecured Deb,08/07/2024,2024-08-14,2024-08-14,A Counsel and Advice,14 IOTA,09-06 - Obtained Advice & Counsel on a Consume...,48,Female,White,< 30% of Area Median Income (AMI),Intake Dept,1,2024,45-54
2,24-0453534,Cheryl,Counts,Closed,01 Bankruptcy/Debtor Relief,010 Bankruptcy: Unsecured Deb,07/10/2024,2024-08-10,2024-08-10,A Counsel and Advice,14 IOTA,09-06 - Obtained Advice & Counsel on a Consume...,56,Female,Black/African American,30 - 49% of AMI,Intake Dept,1,2024,55-64
3,24-0455361,Johnetta,Fisher,Closed,01 Bankruptcy/Debtor Relief,010 Bankruptcy: Unsecured Deb,08/05/2024,NaN,2024-08-06,A Counsel and Advice,14 IOTA,09-06 - Obtained Advice & Counsel on a Consume...,34,Female,Black/African American,30 - 49% of AMI,Intake Dept,4,2024,25-34
4,24-0451164,Tabitha,Scheeley,Closed,01 Bankruptcy/Debtor Relief,010 Bankruptcy: Unsecured Deb,06/10/2024,2024-07-25,2024-07-25,A Counsel and Advice,14 IOTA,09-06 - Obtained Advice & Counsel on a Consume...,30,Female,White,< 30% of Area Median Income (AMI),Intake Dept,4,2024,25-34


In [8]:
# Group by Close Reason and Year
grouped = new_df.groupby(['Date Closed',
                      'Year Closed',
                      'Program Name',
                      'Legal Problem',
                      'Special Legal Problem Code',
                      'Funding Code(s)',
                      'Case Close Reason',
                      'Age Group','Gender',
                      'Race',
                      'Total Number Helped'], 
                      observed=True)['ID#'].count().reset_index()

# Save the data to JSON format
grouped.to_json('data_9-10.json', orient='records')

In [9]:
# Get counts for each age group
age_distribution = new_df['Age Group'].value_counts().sort_index()

# Convert to a dictionary suitable for Chart.js
age_distribution_dict = age_distribution.to_dict()

# Example output for use in Chart.js
print(age_distribution_dict)

{'1-17': 4590, '18-24': 10283, '25-34': 29970, '35-44': 29637, '45-54': 26632, '55-64': 17821, '65-80': 11969, '80+': 2369}


In [10]:
# Group by Close Reason and calculate percentages per year
test_year = 2021  # Example year to filter
df_filtered = new_df[new_df['Year Closed'] == test_year]

In [11]:
# Group by Close Reason
test_close_reason_counts = df_filtered.groupby('Case Close Reason')['ID#'].count().reset_index(name='Count')

# Generate pie chart
fig = px.pie(test_close_reason_counts, values='Count', names='Case Close Reason', title='TEST - Close Reasons for 2021')
fig.show()

In [12]:
# Group by Gender
test_gender_counts = df_filtered.groupby('Gender')['ID#'].count().reset_index(name='Count')

# Generate pie chart
fig = px.pie(test_gender_counts, values='Count', names='Gender', title='TEST - Gender Split for 2021')
fig.show()